In [1]:
# Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPool2D
import sklearn
import cv2
import os
import csv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import PIL
from random import shuffle

/Users/tunc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Global Parameters and Swithes
flip = False
cropSave = False
limitDataSample = True # Not forget to swith off during real training
limitImages = 10
flipSuffixForImage = '_flipped.jpg'
flipSuffixForCSV = '_flipped.csv'
imagePathRoot = "./TrainingData"
croppedImagePathRoot = "./Cropped_Images"
allFileNames = [x for x in os.listdir(imagePathRoot) if  not x.startswith(".")] # Sample_1, Sample_2, Sample_3...
batch_size=32
ep = 5

In [3]:
# Image Flipper
if flip:
    for fileName in allFileNames:
        print("Generating Flipped Images for",fileName)
        singleSamplePath = imagePathRoot +'/'+ fileName
        flippedImagePath = singleSamplePath + '/Flipped_IMG'

        if not os.path.exists(flippedImagePath):
            os.makedirs(flippedImagePath)

        imagePath = singleSamplePath + '/IMG'

        allImageNames = os.listdir(imagePath)
        for imgName in allImageNames:
            image = Image.open(imagePath + '/' + imgName)
            flipImage = image.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            flipImage.save(flippedImagePath+'/'+imgName.replace('.jpg','')+flipSuffixForImage, "JPEG")

    print("Images are flipped and saved successfully..")
else:
    print("Image flipping is not necessary anymore")

Image flipping is not necessary anymore


In [ ]:
#Log Data Flipper
allSamples = []
for fileName in allFileNames:
    singleDrivingLogPath = imagePathRoot +'/'+ fileName +'/'+ "driving_log.csv"
    allLinesFlipped = []
    originalWithOutPaths = []
    with open(singleDrivingLogPath) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            orgLine = line.copy()
            for i in range(0,3): # First three columns are document names and suffix has to be added
                #root = '/'.join(str(path) for path in line[i].split("/")[:-1])
                line[i] = line[i].split("/")[-1].split(".")[0] + flipSuffixForImage
                orgLine[i] = orgLine[i].split("/")[-1].split(".")[0] + ".jpg" 
            steeringAng = line[3] # Here steering is multiplied by -1
            line[3] = str(-1 * float(steeringAng))
            allLinesFlipped.append(line)
            originalWithOutPaths.append(orgLine)
    allSamples = allSamples + (allLinesFlipped + originalWithOutPaths)
    if flip:
        with open(imagePathRoot +'/'+ fileName +'/'+ "driving_log" + flipSuffixForCSV, "w") as csv_file:
            writer = csv.writer(csv_file,delimiter=',')
            for line in allLinesFlipped:
                writer.writerow(line)
        with open(imagePathRoot +'/'+ fileName +'/'+ "driving_log" + "_without_path.csv", "w") as csv_file:
            writer = csv.writer(csv_file,delimiter=',')
            for line in originalWithOutPaths:
                writer.writerow(line)                
if flip:    
    print("Csv Files are flipped and saved successfully..")
else:
    print("Csv Flipping is not necessary anymore")

Csv Flipping is not necessary anymore


In [ ]:
#Image Visualser
#Crop Degrees from:
top = 64
bottom = 22
left = 0
right = 0

images = []
for fileName in allFileNames:
    singleSamplePath = imagePathRoot +'/'+ fileName
    imagePath = singleSamplePath + '/IMG'
    allImageNames = os.listdir(imagePath)
    image = Image.open(imagePath + '/' + allImageNames[0]) # Take first picture from all Samples
    arrIm = np.array(image)
    images.append(arrIm)
    #Crop Image and add after for visualization
    image = image.crop((left, top, arrIm.shape[1]-right, arrIm.shape[0]-bottom)) #(left, top, right, bottom )
    images.append(np.array(image))

%matplotlib inline
fig=plt.figure(figsize=(images[0].shape[0], images[0].shape[1]))
for i in range(1,len(images)+1):
    fig.add_subplot(len(images), 2, i)
    img = images[i-1]
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
#Image Cropper
if cropSave:
    for fileName in allFileNames:
        print(fileName)
        singleSamplePath = imagePathRoot +'/'+ fileName
        folderList = ['/IMG','/Flipped_IMG']
        for eachFolder in folderList:
            imagePath = singleSamplePath + eachFolder
            allImageNames = os.listdir(imagePath)
            for imgName in allImageNames:
                image = Image.open(imagePath + '/' + imgName) # Take first picture from all Samples
                image = image.crop((left, top, arrIm.shape[1]-right, arrIm.shape[0]-bottom)) #(left, top, right, bottom )
                image.save('./Cropped_Images/'+ imgName)
    print("Cropping Finished")
else:
    print("Cropping is not necessary anymore")

In [ ]:
imageDim = images[0].shape # Even index Original 
orgDepth = imageDim[2]
orgHeight = imageDim[0]
orgWidth = imageDim[1]
print("Original Image Dimensions",imageDim)

croppedImageDim = images[1].shape # Odd index Cropped 
cropDepth = croppedImageDim[2]
cropHeight = croppedImageDim[0]
cropWidth = croppedImageDim[1]
print("Cropped Image Dimensions",croppedImageDim)

In [ ]:
class Image:  # Not Used  
    def __init__(self, name, adress,steering):
        self.name = name
        self.adress = adress
        self.steering = steering

In [ ]:
# For sake of Debugging limit the datasample
if limitDataSample:
    allSamples = allSamples[:limitImages]
print("Amount of picture from each view: left, center, right:",len(allSamples)) # All Image Information can be found allSamples

In [ ]:
# Training and Validation Split
allSamplesFilt = [allSamples[i][0:4] for i in range(len(allSamples))] # Get Rid of Unused Features
train_samples, validation_samples = train_test_split(allSamplesFilt, test_size=0.2) # Image Information (left,center,right,) of one frame
num_train_sample = len(train_samples)
num_validation_sample = len(validation_samples)


print("Number of 'Center' Training Images: ", num_train_sample) # (center,left,right,steering)
print("Number of 'Center' Validation Images: ", num_validation_sample)
print("Each Sample Indexing -> train_samples[Index]:")
i = 0
for element in train_samples[0]:
    print("    Index {} : {}".format(i,element))
    i+=1

In [ ]:
# NVIDIA Architecture + My improvements
def myNet(input_shape):

    #Create the Keras Model
    model = Sequential()

    #Crop Images 
    #model.add(Cropping2D(cropping=((top,bottom),(left,right)),input_shape=(orgDepth, orgHeight, orgWidth))) # Original Dimension (depth,height,width)

    #Normalization and Mean center
    model.add(Lambda(lambda x: (x/255.0)-0.5, input_shape = input_shape))

    #Convolutional Layer 1
    model.add(Convolution2D(24,(5,5)))
    model.add(Activation('relu'))

    #Convolutional Layer 2
    model.add(Convolution2D(36,(5,5)))
    model.add(Activation('relu'))

    #Convolutional Layer 3
    model.add(Convolution2D(48,(5,5)))
    model.add(Activation('relu'))

    #Convolutional Layer 4
    model.add(Convolution2D(64,(3,3)))
    model.add(Activation('relu'))

    #Convolutional Layer 5
    model.add(Convolution2D(64,(3,3)))
    model.add(Activation('relu'))

    #Flatten
    model.add(Flatten())

    #Fully Connected Layer 1
    model.add(Dense(100))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    #Fully Connected Layer 2
    model.add(Dense(50))
    model.add(Dropout(0.5)) #TODO ? MAX Pooling?

    #Fully Connected Layer 3
    model.add(Dense(10))
    model.add(Activation('relu'))

    #Final Output
    model.add(Dense(1))

    return model # Can be used for Visualing historyObj.history['loss'] and historyObj.history['loss'] && also check out Keys: print(history_object.history.keys())


In [ ]:
# GENERATOR Function - Keras will use
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = croppedImagePathRoot + "/" + batch_sample[0] # 0 index is Center Image
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3]) # 3 index is Steering
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [ ]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [ ]:
# Validation and Training Seperation
#input_shape = (cropDepth, cropHeight, cropWidth) # Initial Input Image
input_shape = (cropHeight,cropWidth,cropDepth) # Initial Input Image

print(input_shape)
model = myNet(input_shape)
model.compile(loss='mse',optimizer='adam')
historyObj = model.fit_generator(train_generator, steps_per_epoch=num_train_sample,
                        validation_data=validation_generator, steps=num_validation_sample ,
                        epochs = ep, verbose=1)


In [ ]:
print(historyObj.history.keys())

### plot the training and validation loss for each epoch
plt.plot(historyObj.history['loss'])
plt.plot(historyObj.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [ ]:
'''TODOs 
1) Left And Right Image Usage



'''